In [1]:
import sys
sys.path.append("../src")

from utils.datasets_loader import load_datasets
from parsers.argument_classes import DatasetArguments


In [2]:
ROOT_DIR = "../datasets_with_size"
DATASET_NAME = "wtq"

In [3]:
dataset_args = DatasetArguments(
    dataset_root_dir=ROOT_DIR,
    dataset_names=[DATASET_NAME],
    table_extension="csv",
    shuffle=False,
)

In [ ]:
datasets = load_datasets(dataset_args)
print(datasets)

In [ ]:
print(datasets["train"]["id"])

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

In [ ]:
from utils.get_table_size import calculate_tokenized_table_size

def dataset_map(example):
    row_num, width = calculate_tokenized_table_size(example["table"], tokenizer)
    example["table_row_num"] = row_num
    example["table_width"] = width
    return example

datasets["train"] = datasets["train"].map(dataset_map)

In [ ]:
datasets["validation"] = datasets["validation"].map(dataset_map)
datasets["test"] = datasets["test"].map(dataset_map)


In [9]:
# remove table column
datasets["train"] = datasets["train"].remove_columns("table")
datasets["validation"] = datasets["validation"].remove_columns("table")
datasets["test"] = datasets["test"].remove_columns("table")


In [ ]:
datasets

In [ ]:
datasets["test"][0]

In [12]:
df_train = datasets["train"].to_pandas()
df_val = datasets["validation"].to_pandas()
df_test = datasets["test"].to_pandas()

df_train.to_csv(f"{ROOT_DIR}/{DATASET_NAME}/data/train.csv", index=False)
df_val.to_csv(f"{ROOT_DIR}/{DATASET_NAME}/data/val.csv", index=False)
df_test.to_csv(f"{ROOT_DIR}/{DATASET_NAME}/data/test.csv", index=False)